In [42]:
import cv2
import torch
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [43]:
FILE_BASE = "KakaoTalk_20250707_104842383"
VIDEO_PATH = f'dataset/video/{FILE_BASE}.mp4'
OUTPUT_VIDEO_PATH = f'result/video/{FILE_BASE}_result_final.mp4'

In [44]:
tracked_objects = {}
next_object_id = 0
max_static_frames = 10
position_tolerance = 5
tracking_range = 10
frame_id = 0

In [45]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.25

cap = cv2.VideoCapture(VIDEO_PATH)
fgbg = cv2.createBackgroundSubtractorMOG2(history=90, varThreshold=16, detectShadows=False)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

Using cache found in C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-7-8 Python-3.11.13 torch-2.7.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [46]:
def merge_close_boxes(boxes, distance_thresh=30):
    groups = [{i} for i in range(len(boxes))]

    def boxes_are_close(b1, b2, thresh):
        x1, y1, w1, h1 = b1
        x2, y2, w2, h2 = b2
        dx = max(0, max(x1, x2) - min(x1 + w1, x2 + w2))
        dy = max(0, max(y1, y2) - min(y1 + h1, y2 + h2))
        return (dx ** 2 + dy ** 2) ** 0.5 < thresh

    merged = True
    while merged:
        merged = False
        new_groups = []
        skip = set()

        for i in range(len(groups)):
            if i in skip:
                continue
            g1 = groups[i]
            for j in range(i + 1, len(groups)):
                if j in skip:
                    continue
                g2 = groups[j]
                if any(boxes_are_close(boxes[a], boxes[b], distance_thresh) for a in g1 for b in g2):
                    g1 |= g2
                    skip.add(j)
                    merged = True
            new_groups.append(g1)
        groups = new_groups

    merged_boxes = []
    for g in groups:
        xs = [boxes[i][0] for i in g]
        ys = [boxes[i][1] for i in g]
        xes = [boxes[i][0] + boxes[i][2] for i in g]
        yes = [boxes[i][1] + boxes[i][3] for i in g]
        merged_boxes.append((min(xs), min(ys), max(xes) - min(xs), max(yes) - min(ys)))
        
    return merged_boxes

In [47]:
def update_tracked_objects(boxes):
    global tracked_objects, next_object_id

    updated_ids = set()

    for box in boxes:
        x, y, w, h = box
        center = (x + w // 2, y + h // 2)

        matched = False
        for obj_id, data in tracked_objects.items():
            prev_center = data["center"]
            dist = np.linalg.norm(np.array(center) - np.array(prev_center))

            if dist < tracking_range:  # 가까운 객체에 대해서만 비교
                # 🔄 정지 여부 판단
                if dist < position_tolerance:
                    data["static_count"] += 1
                else:
                    data["static_count"] = 0  # 다시 움직임!

                data["center"] = center
                data["last_box"] = box
                updated_ids.add(obj_id)
                matched = True
                break

        if not matched:
            tracked_objects[next_object_id] = {
                "center": center,
                "static_count": 0,
                "last_box": box
            }
            updated_ids.add(next_object_id)
            next_object_id += 1

    # # ❌ 더 이상 관측되지 않은 객체 제거
    # to_delete = [obj_id for obj_id in tracked_objects if obj_id not in updated_ids]
    # for obj_id in to_delete:
    #     del tracked_objects[obj_id]

    return updated_ids

In [48]:
def box_area(box):
    return max(0, box[2] - box[0]) * max(0, box[3] - box[1])

def xywh_box_to_xyxy(x, y, w, h):
    return [x, y, x + w, y + h]

def filter_mog2_by_yolo(yolo_boxes, mog2_boxes, threshold_ratio=0.1):
    from collections import defaultdict

    yolo_dict = defaultdict(list)
    yolo_class_dict = defaultdict(list)

    for box in yolo_boxes:
        frame_id, x, y, w, h, class_name = box
        xyxy = xywh_box_to_xyxy(x, y, w, h)
        yolo_dict[frame_id].append(xyxy)
        yolo_class_dict[frame_id].append((xyxy, class_name))

    result = []

    for box in mog2_boxes:
        frame_id, x, y, w, h = box
        mb = xywh_box_to_xyxy(x, y, w, h)
        m_area = box_area(mb)

        excluded = False

        # 중심점이 airplane YOLO 박스 안에 있는지 확인
        m_x1, m_y1, m_x2, m_y2 = mb
        m_cx = (m_x1 + m_x2) / 2
        m_cy = (m_y1 + m_y2) / 2

        for yb, class_name in yolo_class_dict.get(frame_id, []):
            if class_name == 'airplane':
                y_x1, y_y1, y_x2, y_y2 = yb
                if y_x1 <= m_cx <= y_x2 and y_y1 <= m_cy <= y_y2:
                    excluded = True
                    break

        if not excluded:
            result.append([frame_id, x, y, w, h])

    return result

In [49]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    cutoff_y = int(height * 0.2)
    # cv2.line(frame, (0, cutoff_y), (width, cutoff_y), (255, 0, 0), 2)

    fgmask = fgbg.apply(frame)
    _, binary = cv2.threshold(fgmask, 150, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    results = model(frame)
    yolo_boxes = []
    for *xyxy, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, xyxy)
        w = x2 - x1
        h = y2 - y1
        class_name = model.names[int(cls)]
        if y1 > cutoff_y:
            yolo_boxes.append([frame_id, x1, y1, w, h, class_name])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    mog2_boxes = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 30:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        if y > cutoff_y:
            mog2_boxes.append([frame_id, x, y, w, h])
    
    filtered_boxes = filter_mog2_by_yolo(yolo_boxes, mog2_boxes)
    filtered_xywh = [box[1:] for box in filtered_boxes]
    merged_boxes = merge_close_boxes(filtered_xywh, distance_thresh=5)

    filtered_final = [box[1:] for box in filtered_boxes]
    update_tracked_objects(filtered_final)

    for obj_id, data in tracked_objects.items():
        if data["static_count"] >= max_static_frames:
            x, y, w, h = data["last_box"]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    for i, (x, y, w, h) in enumerate(merged_boxes):
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    out.write(frame)
    cv2.imshow('Frame', frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()